In [4]:
import pandas as pd
import yfinance as yf
import duckdb
from pathlib import Path

### Learning yfinance
Dabbling a bit to see what I'm working with, since I've never used it before and don't know the schema.

In [ ]:
df_test =  yf.download('AAPL', start='2024-01-01', end='2024-12-31', auto_adjust=False, multi_level_index=False).reset_index()# auto_adjust = False gives us Adj Close column, else only gives Close but adjusted
df_test

[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2024-01-02,183.903244,185.639999,188.440002,183.889999,187.149994,82488700
1,2024-01-03,182.526230,184.250000,185.880005,183.429993,184.220001,58414500
2,2024-01-04,180.208130,181.910004,183.089996,180.880005,182.149994,71983600
3,2024-01-05,179.484924,181.179993,182.759995,180.169998,181.990005,62379700
4,2024-01-08,183.823975,185.559998,185.600006,181.500000,182.089996,59144500
...,...,...,...,...,...,...,...
246,2024-12-23,254.120697,255.270004,255.649994,253.449997,254.770004,40858800
247,2024-12-24,257.037506,258.200012,258.209991,255.289993,255.490005,23234700
248,2024-12-26,257.853760,259.019989,260.100006,257.630005,258.190002,27237100
249,2024-12-27,254.439224,255.589996,258.700012,253.059998,257.829987,42355300


In [ ]:
df_test2_tickers = ['AAPL', 'MSFT', 'GOOGL']#, 'AMK'] - Assetmark is "paid release" in Yahoo Finance :(
df_test2 = yf.download(df_test2_tickers, start='2024-01-01', end='2024-12-31', auto_adjust=False, multi_level_index=False).reset_index() # Mutl Level Index False won't work with multiple tickers
df_test2

[*********************100%***********************]  3 of 3 completed


Price        Date   Adj Close                               Close              \
Ticker                   AAPL       GOOGL        MSFT        AAPL       GOOGL   
0      2024-01-02  183.903244  137.133850  365.421600  185.639999  138.169998   
1      2024-01-03  182.526230  137.878235  365.155579  184.250000  138.919998   
2      2024-01-04  180.208130  135.367218  362.534637  181.910004  136.389999   
3      2024-01-05  179.484924  134.712143  362.347412  181.179993  135.729996   
4      2024-01-08  183.823975  137.798828  369.185486  185.559998  138.839996   
..            ...         ...         ...         ...         ...         ...   
246    2024-12-23  254.120697  193.870728  432.062775  255.270004  194.630005   
247    2024-12-24  257.037506  195.344940  436.112885  258.200012  196.110001   
248    2024-12-26  257.853760  194.836945  434.901794  259.019989  195.600006   
249    2024-12-27  254.439224  192.007996  427.377319  255.589996  192.759995   
250    2024-12-30  251.064499  190.493958  421.719025  252.199997  191.240005   

Price                     High                                 Low  \
Ticker        MSFT        AAPL       GOOGL        MSFT        AAPL   
0       370.869995  188.440002  139.449997  375.899994  183.889999   
1       370.600006  185.880005  139.630005  373.260010  183.429993   
2       367.940002  183.089996  139.160004  373.100006  180.880005   
3       367.750000  182.759995  137.160004  372.059998  180.169998   
4       374.690002  185.600006  139.009995  375.200012  181.500000   
..             ...         ...         ...         ...         ...   
246     435.250000  255.649994  195.100006  437.649994  253.449997   
247     439.329987  258.209991  196.110001  439.600006  255.289993   
248     438.109985  260.100006  196.750000  440.940002  257.630005   
249     430.529999  258.700012  195.320007  435.220001  253.059998   
250     424.829987  253.500000  192.550003  427.549988  250.750000   

Price                                 Open                            Volume  \
Ticker       GOOGL        MSFT        AAPL       GOOGL        MSFT      AAPL   
0       136.479996  366.769989  187.149994  138.550003  373.859985  82488700   
1       137.080002  368.510010  184.220001  137.250000  369.010010  58414500   
2       136.350006  367.170013  182.149994  138.419998  370.670013  71983600   
3       135.149994  366.500000  181.990005  136.750000  368.970001  62379700   
4       136.259995  369.010010  182.089996  136.289993  369.299988  59144500   
..             ...         ...         ...         ...         ...       ...   
246     190.149994  432.829987  254.770004  192.619995  436.739990  40858800   
247     193.779999  434.190002  255.490005  194.839996  434.649994  23234700   
248     194.380005  436.630005  258.190002  195.149994  439.079987  27237100   
249     190.649994  426.350006  257.829987  194.949997  434.600006  42355300   
250     189.119995  421.899994  252.229996  189.800003  426.059998  35557500   

Price                       
Ticker     GOOGL      MSFT  
0       23711200  25258600  
1       24212100  23083500  
2       27137700  20901500  
3       22513900  21004600  
4       21404000  23134000  
..           ...       ...  
246     25675000  19152500  
247     10403300   7164500  
248     12046600   8194200  
249     18891400  18117700  
250     14264700  13158700  

[251 rows x 19 columns]

# Compiling dataframe
Looping through each ticker and setting that ticker value in a single column - just sounds easier than unpivoting.

In [23]:
tickers = ['AAPL', 'MSFT', 'GOOGL']
start = '2024-01-01'
end = '2025-09-30'

df_list = []
for t in tickers:
    df = yf.download(t, start=start, end=end, auto_adjust=False, progress=False, multi_level_index=False).reset_index()
    df['ticker'] = t
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)

In [24]:
df

,date,adj_close,close,high,low,open,volume,ticker
0,2024-01-02,183.903214,185.639999,188.440002,183.889999,187.149994,82488700,AAPL
1,2024-01-03,182.526215,184.250000,185.880005,183.429993,184.220001,58414500,AAPL
2,2024-01-04,180.208130,181.910004,183.089996,180.880005,182.149994,71983600,AAPL
3,2024-01-05,179.484955,181.179993,182.759995,180.169998,181.990005,62379700,AAPL
4,2024-01-08,183.823959,185.559998,185.600006,181.500000,182.089996,59144500,AAPL
...,...,...,...,...,...,...,...,...
1306,2025-09-23,251.495499,251.660004,254.360001,250.479996,253.039993,26628000,GOOGL
1307,2025-09-24,246.978455,247.139999,252.350006,246.440002,251.660004,28201000,GOOGL
1308,2025-09-25,245.629318,245.789993,246.490005,240.740005,244.399994,31020400,GOOGL
1309,2025-09-26,246.378830,246.539993,249.419998,245.970001,247.070007,18503200,GOOGL


# Creating the duckdb database file
*.duckdb added to git ignore
DuckDB .WAL (Write Ahead Log) file is created during connection and deleted if closed normally, but remains if crash for data-recovery - cool feature!

In [ ]:
DB_PATH = Path('stocks.duckdb')

con = duckdb.connect(DB_PATH.as_posix()) # As posix to convert to str and adjust for windows paths, path as argument in .connect() creates file to persist data
con.execute('SELECT 1').fetchall() # DuckDB connection has cursor like capabilities, so we can execute and use .fetchone/all/many(x) in one line. con.sql() is also and option, but .execute is more general for other DBs.


[(1,)]

Adding Raw Data as a table

In [26]:
con.execute("CREATE SCHEMA IF NOT EXISTS raw;") # Creating schema
con.register("df_view", df) # Creating in-memory view of dataframe, which we can then persist to table

# Creating table raw.big_3 and persisting view to actual table in duckdb file
con.execute("""
CREATE OR REPLACE TABLE raw.big_3 AS
SELECT
    date::Date as trading_date,
    ticker,
    open,
    high,
    low,
    adj_close,
    close,
    volume
FROM df_view   
""")

In [27]:
con.execute("SELECT COUNT(*) FROM raw.big_3").fetchone()

(1311,)

In [29]:
con.execute("SHOW TABLES;").fetchdf()

,name
0,df_view


In [35]:
con.execute("SELECT * FROM raw.big_3 LIMIT 5;").fetchdf()

,trading_date,ticker,open,high,low,adj_close,close,volume
0,2024-01-02,AAPL,187.149994,188.440002,183.889999,183.903214,185.639999,82488700
1,2024-01-03,AAPL,184.220001,185.880005,183.429993,182.526215,184.250000,58414500
2,2024-01-04,AAPL,182.149994,183.089996,180.880005,180.208130,181.910004,71983600
3,2024-01-05,AAPL,181.990005,182.759995,180.169998,179.484955,181.179993,62379700
4,2024-01-08,AAPL,182.089996,185.600006,181.500000,183.823959,185.559998,59144500


In [32]:
con.close()

# Notes on creating DBT
DBT is installed is a module but it runs SQL in the warehouse (DuckDB in this case). Only local necessity is a profile to connect to DB.
I had to pip install both dbt-core and dbt-duckdb (dbt-duckdb is an adapter. before v1.8, it used to install core too, but not anymore).
Need to initialize dbt project from root of git repo by running "dbt init" in cmd. It will prompt you for project name and DB type. Will create folder with project name and sub folders models, dbt_project.yml and others.


## Manual step: Configuring profiles.yml file
DBT will eiter create it for you or ask you to create it, here on windows: C:\Users\userName\.dbt\profiles.yml. 
Minimal DuckDB profile: 
stocks_dbt:
  target: dev
  outputs:
    dev:
      type: duckdb [other options can be bigquery, postgres, redshift, snowflake, etc]
      path: [path to DB or DuckDB file: stocks.duckdb in this case]
      schema: [default schema dbt writes to, in this case raw. Will default to main if none specified]
      threads: [number of paths (models) dbt will work on at one time. 
                DBT creates DAG when it runs of links between models, threads limit that path. If one, will do one then move to the next one.
                No limit. The more threads, the faster, but more load on warehouse.]
    
    prod: [same info, but obviously different databases if there is a dev and prod environment. Target is set to dev here.]

- DBT created the profile for me, but I still had to edit it and add the schema attribute to the outputs. 

Test connection with "dbt debug" from repo root. Good response is "All Checks Passed" which means DBT is connected to Data Base. 


## Once it's created and passes dbt debug:
1. Create folders for staging and marts in dbt project folder (stocks_dbt, made and init)
  - stocks_dbt/models/[staging, marts]. Just with File Explorer. Just good practice it seems.
2. Declare namespace.table as source (raw.big_3 in this case)
  - A file that tells dbt the table already exists and allows you to reference it with source('raw','big_3'). Also adds tests/docs. Example below:
3. Build staging environment (models/staging/stg_big_3.sql)
  - Cleaning/standardization layer. When DBT runs, it will compile the SQL, wrap it in a CREATE VIEW or CREATE TABLE, executes it in the warehouse (DuckDB) and materializes it as dev.stg_big_3
4. Build Fact Model:
  - Just another statement that uses the view dev.stg_big_3 (via {{ ref(stg_big_3) }}) and adds a couple new columns.
5. Run "dbt run"
  - Tracks dependencies in the Jinja {{ }} and executes SQL within the warehouse.
6. Run "dbt test"
  - Runs tests DBT knows about, like those in the sources.yml file.


# .YML Files:
## profiles.yml
stocks_dbt:
  outputs:
    dev:
      type: duckdb
      path: stocks.duckdb {# full file path in "C:\\Users\\userName\\ ... \\stocks.duckdb" #}
      threads: 1
      schema: raw

    prod:
      type: duckdb
      path: stocks.duckdb {# full file path in "C:\\Users\\userName\\ ... \\stocks.duckdb" #}
      threads: 4
      schema: raw

  target: dev


## sources.yml
version: 2

sources:
  - name: raw
    schema: raw
    tables:
      - name: big_3
        description: "Daily OHLCV prices for AAPL, MSFT, GOOGL from 2024Q1 - 2025Q3 via yfinance"
        columns:
          - name: trading_date
            data_tests: 
              - not_null
          - name: ticker
            data_tests:
              - accepted_values:
                  arguments:
                    values: ["AAPL", "MSFT", "GOOGL"]


# Models/Queries:
**Notes**:
This looks different than SQL I'm used to due to the Jinja (the double curly braces: {{ }}) and DBT Macros (functions inside the {{ }})
 - DBT uses Jinja under the hood, which is a templating language which helps make SQL reusable and tracks dependencies via a DAG. The {{ }}'s tell DBT to elavuate with Jinja before SQL, which allows it to run the macro function inside before execution. 
 - Macros: ref() and source() are used inside the {{ }}.
    - Ref() tells DBT that the fact model is dependent on stg_big_3, and Source() tells DBT that stg_big_3 is dependent on raw.big_3 namespace/tables (basically the DAG), and runs in order of depencies, which already exists and can connect via the profiles.yml we set up.
    - By default, the models are materialized as a table or view in the target schema specified by profiles.yml. So the table of the staging model materializes as dev.stg_big_3 () and ref(stg_big_3) resolves to dev.stg_big_3 because of the active target in the profile, unless overriden using {{ config(schema='staging') }}


### Staging Model (models/staging/stg_big_3.sql) 

select
  trading_date,
  upper(ticker) as ticker,
  cast(open as double) as open,
  cast(high as double) as high,
  cast(low as double) as low,
  cast(close as double) as close,
  cast(adj_close as double) as adj_close,
  cast(volume as bigint) as volume
from {{ source('raw', 'big_3') }}
where trading_date is not null


### Fact Model (Calculates daily Change & Returns for each Company over Date)

with base as (
  select *
  from {{ ref('stg_big_3') }}
),
calc as (
  select
    trading_date,
    ticker,
    close,
    volume,
    close - lag(close) over (partition by ticker order by trading_date) as daily_change,
    (close / lag(close) over (partition by ticker order by trading_date) - 1) as daily_return
  from base
)
select *
from calc

In [ ]:
#con = duckdb.connect("stocks.duckdb")  # use exact same filename/path as profiles.yml
print(con.execute("SHOW TABLES").fetchdf())
print(con.execute("SHOW TABLES FROM raw").fetchdf())

Empty DataFrame
Columns: [name]
Index: []
                  name
0                big_3
1      fct_stock_daily
2   my_first_dbt_model
3  my_second_dbt_model
4            stg_big_3


In [ ]:
fact_df = con.execute("SELECT * FROM raw.fct_stock_daily").df()
fact_df

,trading_date,ticker,close,volume,daily_change,daily_return
0,2024-01-02,MSFT,370.869995,25258600,NaN,NaN
1,2024-01-03,MSFT,370.600006,23083500,-0.269989,-0.000728
2,2024-01-04,MSFT,367.940002,20901500,-2.660004,-0.007178
3,2024-01-05,MSFT,367.750000,21004600,-0.190002,-0.000516
4,2024-01-08,MSFT,374.690002,23134000,6.940002,0.018872
...,...,...,...,...,...,...
1306,2025-09-23,GOOGL,251.660004,26628000,-0.869995,-0.003445
1307,2025-09-24,GOOGL,247.139999,28201000,-4.520004,-0.017961
1308,2025-09-25,GOOGL,245.789993,31020400,-1.350006,-0.005463
1309,2025-09-26,GOOGL,246.539993,18503200,0.750000,0.003051


In [52]:
con.close()

# Let's add a dimension for kicks
Just a small table to house properties of the companies using Ticker as the ID.

Plan: 
 - Add table to sources.yaml (added -name: and description at same indenture as other table below its tests)
 - Just for kicks, we'll add a separate model for it in models\staging (even though we could just join at the end, but good practice)
 - After producing fct_stock_daily, select from it using {{ ref() }} and join dimension table.
 - *Profit*

In [49]:
dict_companies = {
    'company_id':['AAPL', 'MSFT', 'GOOGL'],
    'company_name':['Apple','Microsoft','Google'],
    'ceo':['Tim Cook', 'Satya Nadella', 'Sundar Pichai']
}

df_companies = pd.DataFrame(dict_companies)
df_companies

,company_id,company_name,ceo
0,AAPL,Apple,Tim Cook
1,MSFT,Microsoft,Satya Nadella
2,GOOGL,Google,Sundar Pichai


In [51]:
con.execute("CREATE SCHEMA IF NOT EXISTS raw;") # Creating schema
con.register("df_companies_view", df_companies) # Creating in-memory view of dataframe, which we can then persist to actual table

# Creating table raw.company_dim and persisting view to actual table in duckdb file
con.execute("""
CREATE OR REPLACE TABLE raw.company_dim AS
SELECT
    company_id,
    company_name,
    ceo        
FROM df_companies_view   
""")


# Models:
## Dimension (Model): models\staging\dim_company.sql
select
  upper(company_id) as ticker, -- aliasing to match fact table for join
  company_name,
  ceo
from {{ source('raw', 'company_dim') }}

## Encriched Fact Table (joining dimension): \models\marts\fct_stock_daily_enriched.sql:
select
  f.trading_date,
  f.ticker,
  d.company_name,
  d.ceo,
  f.close,
  f.volume,
  f.daily_change,
  f.daily_return
from {{ ref('fct_stock_daily') }} as f
left join {{ ref('dim_company') }} as d
  on f.ticker = d.ticker


In [58]:
#con = duckdb.connect("stocks.duckdb")  # use exact same filename/path as profiles.yml
print(con.execute("SHOW TABLES").fetchdf())
print(con.execute("SHOW TABLES FROM raw").fetchdf())

Empty DataFrame
Columns: [name]
Index: []
                       name
0                     big_3
1               company_dim
2               dim_company
3           fct_stock_daily
4  fct_stock_daily_enriched
5                 stg_big_3


In [57]:
con.execute("DROP TABLE IF EXISTS raw.my_first_dbt_model;")
con.execute("DROP VIEW IF EXISTS raw.my_second_dbt_model;")

In [59]:
fact_enriched_df = con.execute("SELECT * FROM raw.fct_stock_daily_enriched").df()
fact_enriched_df

,trading_date,ticker,company_name,ceo,close,volume,daily_change,daily_return
0,2024-01-02,MSFT,Microsoft,Satya Nadella,370.869995,25258600,NaN,NaN
1,2024-01-03,MSFT,Microsoft,Satya Nadella,370.600006,23083500,-0.269989,-0.000728
2,2024-01-04,MSFT,Microsoft,Satya Nadella,367.940002,20901500,-2.660004,-0.007178
3,2024-01-05,MSFT,Microsoft,Satya Nadella,367.750000,21004600,-0.190002,-0.000516
4,2024-01-08,MSFT,Microsoft,Satya Nadella,374.690002,23134000,6.940002,0.018872
...,...,...,...,...,...,...,...,...
1306,2025-09-23,GOOGL,Google,Sundar Pichai,251.660004,26628000,-0.869995,-0.003445
1307,2025-09-24,GOOGL,Google,Sundar Pichai,247.139999,28201000,-4.520004,-0.017961
1308,2025-09-25,GOOGL,Google,Sundar Pichai,245.789993,31020400,-1.350006,-0.005463
1309,2025-09-26,GOOGL,Google,Sundar Pichai,246.539993,18503200,0.750000,0.003051


In [60]:
con.close()